In [78]:
#import statements
import pandas as pd
import numpy as np
import os
import re
import time
from langdetect import detect  
from polyglot.detect import Detector 
import cld2

In [79]:
#get relative path
os.listdir(os.getcwd())

['.git',
 '.ipynb_checkpoints',
 'clean_data.csv',
 'clean_data.ipynb',
 'clean_data_cld2withpoly.csv',
 'clean_data_cld2withpoly_7word.csv',
 'clean_data_Cldpoly_0words.csv',
 'clean_data_Cldpoly_3words.csv',
 'clean_data_Cldpoly_4words.csv',
 'clean_data_Cldpoly_no_lencheck.csv',
 'clean_data_poly.csv',
 'README.md',
 'train-balanced.csv',
 'train_balanced.csv']

In [80]:
#read the training data from the csv file
header = ['label','comment']
data = pd.read_table('train-balanced.csv',
                    sep='\t', 
#                     delimiter=',', 
                    names=header,
                    usecols=[0,1],
                    dtype={'label':int,'comment':str},
                    keep_default_na=False)

In [81]:
#helper function to clean the comments
def comment_clean(user_comment):
    # remove trailing \r and \n    
    user_comment.rstrip('\r\n')
    
    #remove the # from hashtag
    if '#' in user_comment:
        hash_tag = re.search('#',user_comment)
        if hash_tag is not None:
            user_comment = user_comment.replace(hash_tag.group(0),' ')
    #remove the redit tags(r/) from comment
    if 'r/' in user_comment:
        r_tag = re.search('r/',user_comment)
        if r_tag is not None:
            user_comment = user_comment.replace(r_tag.group(0),' ')
    #remove the URL links from comments  
    if 'HTTP' in user_comment:
        # url of the form [link name](http://url)
        url_link = re.search('\[(.*)\(HTTP(.*)\)', user_comment)
        if url_link is not None:
            user_comment = user_comment.replace(url_link.group(0),' ')
    if 'http' in user_comment:
        # url of the form [link name](http://url)
        url_link = re.search('\[(.*)\(http(.*)\)', user_comment)
        if url_link is not None:
            user_comment = user_comment.replace(url_link.group(0),' ')
        else:
            #url of the form http:/
            url_link = re.search('http(.*)', user_comment)
            if url_link is not None:
                user_comment = user_comment.replace(url_link.group(0),' ') 
                
    # remove numbers from comments
    user_comment_not_num = re.sub(r'\d+', '', user_comment)   
        
    # replace non english comments with empty string
    
    isReliable, textBytesFound, details = cld2.detect(user_comment_not_num)
    cld_match = details[0][0]
    if not (cld_match == 'ENGLISH'):
        poly_match = Detector(user_comment_not_num, quiet=True).language.name
        if (poly_match != 'English'):
            user_comment = ' '               
   
            
    return user_comment           

In [82]:
#clean each commentnon_eng_data = []
start_time = time.time()
data['comment'] = data.comment.apply(comment_clean)
# remove data with empty comments
valid_comment = data['comment'] != ' '
data = data[valid_comment]
end_time = time.time()
print("time taken ", end_time-start_time)

time taken  1.583024263381958


In [83]:
#write the cleaned data into a csv file
data.to_csv('clean_data_Cldpoly.csv',
           sep= '|',
           index=False)